In [1]:
from dotenv import load_dotenv

In [2]:
load_dotenv()

True

In [3]:
from langchain_teddynote import logging
logging.langsmith("lunch-lens", "0.1.0")

LangSmith 추적을 시작합니다.
[프로젝트명]
lunch-lens


In [6]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import PydanticOutputParser
from 

In [7]:
llm = ChatOpenAI(temperature=0.0)

In [69]:
from pydantic import BaseModel, Field

class FoodInfo(BaseModel):
    name: str = Field(description="The main dish name")
    category: str = Field(description="The category of the main dish")

class FoodInfoList(BaseModel):
    food_list: list[dict[str, str]] = Field(description="The list of main dish names and their categories")

In [70]:
parser = PydanticOutputParser(pydantic_object=FoodInfoList)

In [113]:
from langchain_core.prompts import load_prompt

prompt = load_prompt("./dags/prompts/categorize_food.yaml").partial(format_instructions=parser.get_format_instructions())

In [122]:
name_list = [
    "김치찌개", "된장찌개", "파스타", "스테이크", "가츠동", "돈까스", "떡볶이", "스테이크", 
    "오이무침", "김치볶음밥", "잡채", "갈비찜", "삼겹살", "초밥", "라멘", "짜장면", 
    "짬뽕", "탕수육", "마파두부", "육회", "샤브샤브", "해물파전", "비빔밥", "콩나물국", 
    "불고기", "생선구이", "회덮밥", "물냉면", "비빔냉면", "칼국수", "순두부찌개", 
    "청국장", "깐풍기", "닭갈비", "치킨", "감자탕", "삼계탕", "떡국", "라볶이", 
    "호떡", "닭강정", "부대찌개", "닭볶음탕", "육개장", "아구찜", "매운탕", "감자전", 
    "호박전", "김밥", "참치마요덮밥", "제육볶음", "차돌박이", "돼지갈비", "족발", 
    "보쌈", "막국수", "우동", "가라아게", "오므라이스", "햄버거", "치즈피자", 
    "페퍼로니피자", "핫도그", "케밥", "도넛", "타코", "초콜릿케이크", "치즈케이크", 
    "푸딩", "카레라이스", "야키소바", "오코노미야키", "돈부리", "차슈라멘", "훈제오리", 
    "규동", "치킨커리", "감바스", "크림파스타", "리조또", "토스트", "바게트", 
    "딤섬", "중화비빔밥", "오징어볶음", "코다리찜", "꼬막비빔밥", "황태구이", 
    "양념게장", "간장게장", "쭈꾸미볶음", "고추장불고기", "깻잎김치", "달걀말이", 
    "샌드위치", "브라우니", "푸아그라", "뇨끼", "라자냐", "coke"
]

In [123]:
print(prompt.format(name_list=name_list, count=len(name_list)))

Given the 100 number of recipe titles, for each title,please:
1. Remove any unnecessary descriptive words and return just the main dish name
2. Classify the dish into one of these categories: 한식, 양식, 중식, 일식, 기타
3. Return the main dish name and its category
4. Aggregate all the main dish names and their categories into a list
5. If there are duplicates in the list, only return the first one

Example:
Input: ["김치찌개", "된장찌개", "파스타", "스테이크", "가츠동", "돈까스", "백종원의 초간단 매콤 닭볶음탕", "김치 갈비찜"]
then should return: {"food_list": [{"name": "김치찌개", "category": "한식"}, {"name": "된장찌개", "category": "한식"}, {"name": "파스타", "category": "한식"}, {"name": "스테이크", "category": "한식"}, {"name": "가츠동", "category": "한식"}, {"name": "돈까스", "category": "한식"}, {"name": "백종원의 초간단 매콤 닭볶음탕", "category": "한식"}, {"name": "김치 갈비찜", "category": "한식"}]}

Format:
The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "descript

In [124]:
chain = (prompt | llm | parser)

In [133]:
response = chain.invoke({"name_list":name_list[:50], "count": len(name_list[:50])})

In [126]:
len(name_list)

100

In [127]:
len(response.food_list)

98

In [134]:
response_names = [food['name'] for food in response.food_list]

In [135]:
set(name_list[:50]) - set(response_names)

set()

In [136]:
response.food_list

[{'name': '김치찌개', 'category': '한식'},
 {'name': '된장찌개', 'category': '한식'},
 {'name': '파스타', 'category': '양식'},
 {'name': '스테이크', 'category': '양식'},
 {'name': '가츠동', 'category': '일식'},
 {'name': '돈까스', 'category': '일식'},
 {'name': '떡볶이', 'category': '한식'},
 {'name': '오이무침', 'category': '한식'},
 {'name': '김치볶음밥', 'category': '한식'},
 {'name': '잡채', 'category': '한식'},
 {'name': '갈비찜', 'category': '한식'},
 {'name': '삼겹살', 'category': '한식'},
 {'name': '초밥', 'category': '일식'},
 {'name': '라멘', 'category': '일식'},
 {'name': '짜장면', 'category': '중식'},
 {'name': '짬뽕', 'category': '중식'},
 {'name': '탕수육', 'category': '중식'},
 {'name': '마파두부', 'category': '중식'},
 {'name': '육회', 'category': '한식'},
 {'name': '샤브샤브', 'category': '한식'},
 {'name': '해물파전', 'category': '한식'},
 {'name': '비빔밥', 'category': '한식'},
 {'name': '콩나물국', 'category': '한식'},
 {'name': '불고기', 'category': '한식'},
 {'name': '생선구이', 'category': '한식'},
 {'name': '회덮밥', 'category': '한식'},
 {'name': '물냉면', 'category': '한식'},
 {'name': '비빔냉면', 'cat